In [ ]:
pip install rasterio # type: ignore;

: 

In [ ]:
pip install arcpy;

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement arcpy (from versions: none)
ERROR: No matching distribution found for arcpy


In [ ]:
import os
import numpy as np
import pandas as pd
import rasterio

# 제주 Hillshade 및 Output 디렉토리 설정
hillshade_dir = r"E:\2025_Jeju_SectorCoupling\SolarArea_Jeju"
output_dir = r"E:\2025_Jeju_SectorCoupling\PVgen_Jeju"
csv_path = r"E:\2025_Jeju_SectorCoupling\Jeju_Solar.csv"

# Hillshade 파일 목록 가져오기
raster_files = [f for f in os.listdir(hillshade_dir) if f.endswith('.tif')]

# CSV 파일 읽기
df = pd.read_csv(csv_path, encoding="utf-8", na_values=["", "NA", "null"])

# 파일 개수 확인
if len(raster_files) != len(df):
    print("Warning: Hillshade 파일 개수와 CSV 행 개수가 일치하지 않습니다!")
    print(f"Hillshade 파일 수: {len(raster_files)}, CSV 행 수: {len(df)}")

# Hillshade와 CSV 데이터를 순차적으로 매핑하여 발전량 계산
for raster_file, (_, row) in zip(raster_files, df.iterrows()):
    irradiance = row[1]  # CSV 두 번째 열의 일사량 데이터
    
    # Hillshade 파일 경로
    raster_path = os.path.join(hillshade_dir, raster_file)
    
    with rasterio.open(raster_path) as src:
        raster_data = src.read(1)
        
        # Hillshade Factor 계산
        hillshade_factor = np.where(np.isnan(raster_data) | (raster_data == 0), 0, 1)
        
        # 발전량 계산
        solar_generation = 0.22 * irradiance * hillshade_factor * 900
        
        # Output 파일 저장 경로 및 이름 설정
        output_filename = f"PVgen_Jeju_{raster_file}"  # 파일 이름 재사용
        output_path = os.path.join(output_dir, output_filename)
        
        # 결과 저장
        with rasterio.open(output_path, 'w', driver='GTiff',
                           count=1, dtype=solar_generation.dtype,
                           crs=src.crs, transform=src.transform,
                           width=src.width, height=src.height) as dst:
            dst.write(solar_generation, 1)
        
        print(f"파일이 저장되었습니다: {output_path}")
